In [1]:
#dependencies and set up
import pandas as pd
import os

#Loading the CSV file
device_adoption = pd.read_csv("~/Desktop/Simulated_device_adoption/device_adoption_data.csv")

#Review the DataFrame
device_adoption.head()

,Device_ID,Procedure_Date,Physician_ID,Hospital_System_ID,Geographic_Region,Specialty,Target_Patient_Cohort,Revenue_Impact
0,25795,2024-03-08,9125,424,East,Orthopedics,Standard,41750.75
1,10860,2025-08-07,4291,491,West,Cardiology,High-Risk,59147.77
2,86820,2023-12-26,5033,787,West,Cardiology,Standard,57420.88
3,64886,2025-04-03,6105,727,East,Orthopedics,Standard,50666.11
4,16265,2022-01-05,1703,539,Midwest,Orthopedics,Standard,54107.86


In [2]:
# ---Physician Adoption Growth Rate (Month-to-Month)---

In [3]:
#convert Procedure_Date to month-year format
device_adoption['Procedure_Date'] = pd.to_datetime(device_adoption['Procedure_Date'])
print("\n 'Procedure_Date' successfully converted to datetime.")

device_adoption['Month'] = device_adoption['Procedure_Date'].dt.to_period('M')
print ("\n ''Month' column created using .dt.to_period('M').")
print(device_adoption[['Procedure_Date', 'Month']].head())


 'Procedure_Date' successfully converted to datetime.

 ''Month' column created using .dt.to_period('M').
  Procedure_Date    Month
0     2024-03-08  2024-03
1     2025-08-07  2025-08
2     2023-12-26  2023-12
3     2025-04-03  2025-04
4     2022-01-05  2022-01


In [17]:
#Count uniqe physicians adopting devices per month
physician_growth = (
    device_adoption.groupby('Month')['Physician_ID']
    .nunique()
    .reset_index(name='Unique_Physicians')
)

#Display result
print(physician_growth[['Unique_Physicians', 'Month']].head())

   Unique_Physicians    Month
0                 35  2022-01
1                 21  2022-02
2                 21  2022-03
3                 20  2022-04
4                 26  2022-05


In [5]:
#Calculate Month-to-Month growth rate
physician_growth['MoM_Growth_%'] = (
    physician_growth['Unique_Physicians']
    .pct_change() * 100
).round(2)

#Display result
physician_growth.head(12)

,Month,Unique_Physicians,MoM_Growth_%
0,2022-01,35,NaN
1,2022-02,21,-40.00
2,2022-03,21,0.00
3,2022-04,20,-4.76
4,2022-05,26,30.00
5,2022-06,16,-38.46
6,2022-07,20,25.00
7,2022-08,19,-5.00
8,2022-09,12,-36.84
9,2022-10,22,83.33


In [6]:
#Period,Unique Physicians,MoM Growth (%),Interpretation (Actionable Insight)
#2022-02,21,-40.00,"Significant contraction in new physician adoption, indicating a possible slowdown in initial sales/marketing efforts post-launch."
#2022-05,26,+30.00,"A strong rebound, suggesting a successful marketing campaign or sales initiative was executed in April/May."
#2022-10,22,+83.33,"The highest growth rate in the year. This is a critical month to investigate. The company should analyze which new initiatives (e.g., new territory sales rep, clinical trial results, or a medical conference) drove this dramatic acceleration to replicate the success."
#Overall Trend,Varies,Fluctuates Wildly,"Adoption is highly volatile, suggesting sales efforts are episodic or reactive rather than consistent. The customer should investigate the root cause of the steep drops (e.g., in June, September) to stabilize their commercial strategy."

In [7]:
# ---Revenue Concentration (Top 5 Hospital Systems)---

In [13]:
#Calculate total revenue per hospital system
revenue_by_hospital = (
    device_adoption.groupby('Hospital_System_ID')['Revenue_Impact']
    .sum()
    .reset_index()
    .sort_values(by='Revenue_Impact', ascending= False)
)

#Top 5 Hospital Systems
top5_hospitals = revenue_by_hospital.head(5)

revenue_by_hospital.head(5)

,Hospital_System_ID,Revenue_Impact
559,932,310935.01
69,208,307475.20
353,632,287915.20
390,681,276066.29
229,453,240674.05


In [14]:
#Calculate their share of total revenue
total_revenue = revenue_by_hospital['Revenue_Impact'].sum()
top5_hospitals['Revenue_Share_%'] = (
    top5_hospitals['Revenue_Impact']/total_revenue *  100
).round(2)

#Display result
top5_hospitals

/var/folders/0c/p9kp7cbj3xqf374gz23y9mnm0000gn/T/ipykernel_16903/1083561493.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top5_hospitals['Revenue_Share_%'] = (


,Hospital_System_ID,Revenue_Impact,Revenue_Share_%
559,932,310935.01,0.62
69,208,307475.20,0.62
353,632,287915.20,0.58
390,681,276066.29,0.55
229,453,240674.05,0.48


In [ ]:
#Hospital_System_ID,Revenue_Impact,Revenue_Share_%,Interpretation
#932,"$310,935.01",0.62%,"This hospital is the single largest revenue source, accounting for 0.62% of the total revenue."
#208,"$307,475.20",0.62%,"Closely following the top performer, this system also accounts for 0.62% of the revenue."
#632,"$287,915.20",0.58%,"This system ranks third, contributing slightly less to the overall revenue."
#681,"$276,066.29",0.55%,Ranks fourth among the top accounts.
#453,"$240,674.05",0.48%,"The fifth-largest system, generating nearly a quarter million in revenue."

In [ ]:
#Key Strategic Insights for the CustomerLow Revenue Concentration: The most significant observation is the low percentage share of the top accounts.If you sum the shares ($0.62 + 0.62 + 0.58 + 0.55 + 0.48$), the Top 5 Hospitals only account for $\approx 2.85\%$ of the total market revenue.Actionable Insight: The customer is not overly reliant on a few huge accounts. The sales strategy should focus heavily on market breadth and efficiency (scalability, as mentioned in the job description) rather than deep customization for a handful of giant accounts. They need to find repeatable processes to activate and grow many smaller accounts.Benchmark for Success: Hospital System 932 sets the revenue benchmark at $\approx \$310k$.Actionable Insight: The customer should investigate the successful implementation model (e.g., number of adopting physicians, types of patients, sales strategy) used at System 932 and standardize it to uplift the performance of other hospitals that are lagging.Tiered Targeting: These Top 5 accounts are the Tier 1 targets and should receive focused relationship management to ensure contract renewal and prevent competitive penetration, even though their individual share is small. The majority of the company's effort, however, must be spread across the broader base of hospitals.

In [ ]:
# ---Procedure Breakdown by Specialty and Patient Cohort ---

In [15]:
#Group and summarize
procedure_breakdown = (
    device_adoption.groupby(['Specialty', 'Target_Patient_Cohort'])
    .agg(
        Total_Procedures=('Device_ID', 'count'),
        Avg_Revenue=('Revenue_Impact', 'mean')
    )
    .reset_index()
    .sort_values(by='Total_Procedures', ascending=False)
)

#Display result
procedure_breakdown

,Specialty,Target_Patient_Cohort,Total_Procedures,Avg_Revenue
1,Cardiology,Standard,312,51125.068109
5,Orthopedics,Standard,240,48546.841792
0,Cardiology,High-Risk,145,49253.645103
3,General Surgery,Standard,119,50633.275126
4,Orthopedics,High-Risk,113,50260.979735
2,General Surgery,High-Risk,71,48253.678592


In [ ]:
#Key Strategic Insights for the Customer
#1. Market Traction (Where Usage is Highest)
#Cardiology Dominates: The Cardiology specialty accounts for the highest volume of procedures overall (312 Standard + 145 High-Risk = 457 total procedures), indicating the device has found its strongest product-market fit here.

#Standard Cohort Preference: Across all specialties, the device is used significantly more often in Standard patient cohorts than in High-Risk cohorts (e.g., Cardiology has 312 Standard vs. 145 High-Risk procedures).

#Actionable Insight: The customer should investigate if this is due to lack of physician training/confidence to use the device on complex, high-risk cases, or if the device's clinical benefits are simply better suited for standard-risk patients. Targeted training for high-risk procedures could expand the market.

#2. Revenue Performance (Commercial Value)
#Revenue is Consistent: The Avg_Revenue per procedure is remarkably consistent across all segments, clustering around $48,000 to $51,000.

#Actionable Insight: Pricing and reimbursement appear stable regardless of the specialty or patient cohort. The sales team should focus on volume (Total_Procedures), as average revenue doesn't vary enough to justify prioritizing one group over another based on price alone.

#3. Lowest Penetration (Target for Growth)
#General Surgery's High-Risk Cohort has the lowest procedure volume (71).

#Actionable Insight: This is the segment with the most untapped potential. If the device is clinically appropriate, the customer should launch a focused marketing campaign and specialized training to increase physician adoption within the General Surgery community, especially for complex, High-Risk cases.